In [64]:
import numpy as np
import random

In [65]:
def gradient_descent(gradient_func, start_point, iterations, eps):
    '''
    Метод градиентного спуска
    :param gradient_func: функция градиента
    :param step_func: функция изменения шага
    :param start_point: начальное приближение
    :param eps: точность расчета
    '''
    current_point = start_point
    current_gradient = np.array([0.0] * len(start_point))
    for it in range(iterations):
        next_gradient = gradient_func(current_point, current_gradient)
        next_point = current_point + next_gradient
        distance = np.linalg.norm(current_point - next_point)

        if distance < eps:
            return current_point, it

        current_point = next_point
        current_gradient = next_gradient

    return current_point, iterations

In [66]:
class MeanSquaredError:
    def function(self, regression, points, state):
        sum_square_error = 0.0
        for p in points:
            sum_square_error += (p[1] - regression.function(state, p[0])) ** 2
        return sum_square_error / len(points)

    def gradient(self, regression, points, state):
        sum_square_error = np.array([0.0] * len(state + 1))
        for p in points:
            sum_square_error -= 2 * (p[1] - regression.function(state, p[0])) * regression.gradient(state, p[0])
        return sum_square_error / len(points)

In [70]:
def standart_gradient(regression, points, n, error_func, step):
    def next_gradient(current_point, current_gradient):
        return - step * error_func.gradient(
            regression, random.sample(points, n), current_point)
    return next_gradient

def momentum_gradient(regression, points, n, error_func, mu, step):
    def next_gradient(current_point, current_gradient):
        return mu * current_gradient - step * error_func.gradient(
            regression, random.sample(points, n), current_point)
    return next_gradient

def nesterov_gradient(regression, points, n, error_func, mu, step):
    def next_gradient(current_point, current_gradient):
        return mu * current_gradient - step * error_func.gradient(
            regression, random.sample(points, n), current_point + mu * current_gradient)
    return next_gradient

In [68]:
class LinearRegression:
    def function(self, state, point):
        res = state[0]
        for i in range(len(point)):
            res += state[i + 1] * point[i]
        return res

    def gradient(self, state, point):
        return np.concatenate(([1.0], point))

In [72]:
points = [
          ([0.0], 0.8),
          ([1.0], 1.1),
          ([2.6], 0.2),
          ([-1.3], 5.0)
]

points_2 = [
          ([0.0], -1.0),
          ([3.0], 8.0),
          ([-1.0], -4.0)
]

print(gradient_descent(
    gradient_func=standart_gradient(
        regression=LinearRegression(),
        points=points_2,
        n=1,
        error_func=MeanSquaredError(),
        step=0.1
    ),
    start_point=np.array([1.0, 2.0]), 
    iterations=200,
    eps=1e-5
))

print(gradient_descent(
    gradient_func=momentum_gradient(
        regression=LinearRegression(),
        points=points_2,
        n=3,
        error_func=MeanSquaredError(),
        step=0.1,
        mu=0.9
    ),
    start_point=np.array([1.0, 2.0]), 
    iterations=200,
    eps=1e-5
))

print(gradient_descent(
    gradient_func=nesterov_gradient(
        regression=LinearRegression(),
        points=points_2,
        n=3,
        error_func=MeanSquaredError(),
        step=0.1,
        mu=0.9
    ),
    start_point=np.array([1.0, 2.0]), 
    iterations=200,
    eps=1e-5
))

(array([-1.00003598,  3.00057618]), 181)
(array([-1.00000255,  3.00000574]), 200)
(array([-1.0010936 ,  3.00029042]), 52)
